In [1]:
using Distributions
using Plots
using LaTeXStrings
using LinearAlgebra
using Colors
using JLD
using Parameters
using Random
import ProgressMeter
using AIDA

┌ Info: Precompiling AIDA [02ab3c64-7f6e-4624-92b1-4056b28faae1]
└ @ Base loading.jl:1317
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for thi

These notebooks beg more explanations.

### EFE Agent

In [3]:
target = [0.8, 0.2]

ndims = 2
npoints = 1

x1 = rand(ndims, npoints) #.* 5
y1 = generate_user_response.(eachcol(x1))

n_steps = 50
gridman = LinRange(0, 1, n_steps)
σ = 0.5
l = 0.5
bro = Iterators.product(gridman, gridman)

# Keep track of last point visited
current = (0.5, 0.5)
grid = Iterators.product(gridman, gridman)

Base.Iterators.ProductIterator{Tuple{LinRange{Float64}, LinRange{Float64}}}((range(0.0, stop=1.0, length=50), range(0.0, stop=1.0, length=50)))

In [4]:
T = 10
p = ProgressMeter.Progress(T, 1)
let x1 = x1, y1 = y1, current = current, σ = σ, l = l
    anim = @animate for t ∈ 1:T
        x2, value_grid = get_new_proposal(grid, x1, y1, current, σ, l)

        # Update current point after testing
        current = (x2[1], x2[2])

        # Get some user feedback
        y1 = vcat(y1, generate_user_response(collect(x2)))
        x1 = hcat(x1, collect(x2))

        # Optimize hyperparams every 5th iteration
        if t % 5 == 0
            σ, l = optimize_hyperparams(x1, y1, [σ, l])
        end

        heatmap(gridman, gridman, value_grid)
        scatter!([x2[2]], [x2[1]], markersize = 10, label = "Current") # This gets flipped for some reason???
        scatter!([target[2]], [target[1]], markersize = 10, label = "Target")
        ProgressMeter.next!(p)
    end

    gif(anim, "tmp/test.gif", fps = 2)
end;

Progress: 100%|█████████████████████████████████████████| Time: 0:00:22
┌ Info: Saved animation to 
│   fn = /Users/apodusenko/Documents/Julia/AIDA/demo/verification/tmp/test.gif
└ @ Plots /Users/apodusenko/.julia/packages/Plots/1RWWg/src/animation.jl:114


### Preference learning

In [46]:
target = [0.8, 0.2]

# Save figures?
savefig_ = false

T = 80
ndims = 2
npoints = 1

# There's probably a better way than gridsearch
n_steps = 50
gridman = LinRange(0, 1, n_steps)
σ = 0.5
l = 0.5

# Keep track of last point visited
current = (0.5, 0.5)
grid = Iterators.product(gridman, gridman)
# Initialise at a random point
x1 = rand(ndims, npoints) #.* 5
y1 = generate_user_response.(eachcol(x1))

# bookkeeping
efe_grids, epi_grids, inst_grids, idxs = repeat([[]], 4);

In [47]:
p = ProgressMeter.Progress(T, 1)
let x1 = x1, y1 = y1, current = current, σ = σ, l = l, idxs = idxs
    for t ∈ 1:T
        x2, epi_grid, inst_grid, value_grid, idx = get_new_decomp(grid, x1, y1, current, σ, l)

        # Update current point after testing
        current = (x2[1], x2[2])

        # Get some user feedback
        y1 = vcat(y1, generate_user_response(collect(x2)))
        x1 = hcat(x1, collect(x2))

        # Optimize hyperparams every 5th iteration
        if t % 5 == 0
            σ, l = optimize_hyperparams(x1, y1, [σ, l])
        end

        append!(efe_grids, value_grid)
        append!(epi_grids, epi_grid)
        append!(inst_grids, inst_grid)
        idxs = vcat(idxs, idx)

        if savefig_
            heatmap(gridman, gridman, value_grid)
            scatter!([x2[2]], [x2[1]], markersize = 10, label = "Current") # This gets flipped for some reason???
            scatter!([target[2]], [target[1]], markersize = 10, label = "Target")
            savefig("tmp/timepoint_" * (t < 10 ? "0" * repr(t) : repr(t)))
        end
        ProgressMeter.next!(p)
    end
    println("All done, saving results")
    save("jlds/experiment.jld", "efe_grids", efe_grids, "epi_grids", epi_grids, "inst_grids", inst_grids, "responses", y1, "points", x1, "idxs", idxs)
end;

Progress: 100%|█████████████████████████████████████████| Time: 0:08:24


All done, saving results


## PGFPLOTS

In [40]:
using PGFPlotsX
pgfplotsx()
push!(PGFPlotsX.CUSTOM_PREAMBLE, raw"\usepgfplotslibrary{fillbetween}");

In [41]:
T = 80
n_steps = 50
gridman = LinRange(0, 1, n_steps);

### Heatmap

In [42]:
experiment = JLD.load("jlds/experiment.jld")

efe_grids = experiment["efe_grids"]
efe_grids = reshape(efe_grids, n_steps,n_steps, T);
efe_grids[efe_grids .== Inf] .= maximum(efe_grids[efe_grids .!= Inf])

epi_grids = experiment["epi_grids"]
epi_grids = reshape(epi_grids, n_steps,n_steps, T);

inst_grids = experiment["inst_grids"]
inst_grids = reshape(inst_grids, n_steps,n_steps, T);

idxs = experiment["idxs"]
points = experiment["points"];

In [44]:
index = 80
x = y = gridman

target = [[0.8], [0.2]]
current = map(x -> [first(x)], points[:,index])


axis1 = @pgf Axis(
    {
        title="EFE landscape at time $(index)",
        xlabel=L"g_s",
        ylabel=L"g_n",
        zlabel="hz",
        legend_pos = "north east",
        legend_cell_align="{left}",
        grid = "major",
        view = (0, 90),
        colorbar,
        "colormap/hot",
        colorbar_style={"xlabel=EFE, xticklabel pos=upper"}
    },
    Plot({only_marks, color="green", mark_size=3.0, mark="*"}, 
    Coordinates(target...)), LegendEntry("Target"),
    Plot({only_marks, color="black", mark_size=3.0, mark="*"}, 
    Coordinates(current...)), LegendEntry("Current"),
    Plot3(
        {
            surf,
            shader = "flat",
        },
        Coordinates(x, y, efe_grids[:, :, index]))
    )
pgfsave("tmp/heatmap_$(index).pdf", axis1)
pgfsave("tikz/heatmap_$(index).tikz", axis1)

### EFE plots

In [83]:
# Get the values of queried points for all time steps
epi_mins = [epi_grids[:,:,i][idxs[i]] for i in 1:T]
inst_mins = [inst_grids[:,:,i][idxs[i]] for i in 1:T]

# Get rid of infs for inhibition of return
epi_means = [mean(epi_grids[:,:,i][isfinite.(epi_grids[:,:,i])]) for i in 1:T]
inst_means = [mean(inst_grids[:,:,i][isfinite.(inst_grids[:,:,i])]) for i in 1:T];

In [84]:
plt_agent = @pgf GroupPlot(
    # group plot options
    {
        width="15cm", height="8cm",

        group_style = {
            group_size = "2 by 1",
            horizontal_sep = "1.5cm",
        },
    },
    # first
    {xlabel=L"t",
           ylabel="value",
           legend_pos = "north east",
           legend_cell_align="{left}",
           grid = "major",
        #    title="TODO: TITLE",
           legend_style = "{nodes={scale=0.8, transform shape}}",
    },
    Plot(
        {no_marks,color="blue"}, Coordinates(collect(1:T), epi_mins)
    ), LegendEntry("Epistemic value"),
    # second
    {xlabel=L"t",
           legend_pos = "south east",
           legend_cell_align="{left}",
           grid = "major",
        #    title="Dataset $(index)",
           legend_style = "{nodes={scale=0.8, transform shape}}",
    },
    Plot(
        {no_marks,color="blue"}, Coordinates(collect(1:T), inst_mins)
    ), LegendEntry("Instrumental value"),
)

pgfsave("tmp/grouped_agent.pdf", plt_agent)
pgfsave("tikz/grouped_agent.tikz", plt_agent)

In [85]:
# Plot of user pref function
include("../../src/environment/user.jl")
n_ticks = 100 # How finely to grid the preference function

x = y = LinRange(0,1,n_ticks)

z = zeros(n_ticks,n_ticks);
for i in 1:n_ticks
    for j in 1:100
	z[i,j] = generate_user_response([i/n_ticks,j/n_ticks],binary=false)
    end
end;

pref_map= @pgf Axis(
    {
        title="User preference function",
        xlabel=L"g_s",
        ylabel=L"g_n",
        zlabel="hz",
        legend_pos = "north east",
        legend_cell_align="{left}",
        grid = "major",
        view = (0, 90),
        colorbar,
        "colormap/hot",
        colorbar_style={"xlabel=θ, xticklabel pos=upper"}
    },
    Plot3(
        {
            surf,
            shader = "flat",
        },
        Coordinates(x, y, z))
    )
pgfsave("tmp/pref_map.pdf", pref_map)
pgfsave("tikz/pref_map.tikz", pref_map)

### Batch Plots

In [86]:
using Colors
experiment_1 = JLD.load("batch_experiment_1.jld")
experiment_2 = JLD.load("batch_experiment_2.jld")

# Heatmap of positive user appraisals
responses_1 = experiment_1["responses"]
responses_2 = experiment_2["responses"]
resp = vcat(responses_1,responses_2);

# Histogram of first index of positive response
idxs = [findfirst( isequal(1),resp[i,:]) for i in 1:size(resp)[1]]
idxs[isnothing.(idxs)] .= 82 # If the agent didn't get a thumbsup, set it to aft;

In [87]:
y = 1:T 
x = 1:T+1

resp_map= @pgf Axis(
    {
        title="Positive Appraisals",
        xlabel="Time Index",
        ylabel="Agent number",
        grid = "minor",
        view = (0, 90), 
        colormap = "{blackwhite}{color=(black) color=(yellow)}"
    },
    Plot3(
        {
            surf,
            shader = "flat corner",
        },
        Coordinates(x, y, resp')),
    )
pgfsave("tmp/resp_map.pdf", resp_map)
pgfsave("tikz/resp_map.tikz", resp_map)

In [88]:
using StatsBase: Histogram, fit, normalize
h = fit(Histogram, Float64.(idxs), nbins=100, closed = :left)

shift = 2
plt_histogram = @pgf Axis(
    {
        xlabel="\\small{Time Index}",
        ylabel="\\small{Number of Postive responses}",
        title="Ensemble Performance",
        ticklabel_style={"font=\\tiny"},
        yticklabel_style={
        "/pgf/number format/fixed,
        /pgf/number format/precision=0"
        },
        grid="major",
        yminorgrids=true,
        tick_align="outside",
        xtick=range(0, 80; length=5),
        ytick=range(1, 18; length=9),
        xmin =0.0,
        xmax=maximum(idxs)+shift,
        scaled_y_ticks = false,
    },
    Plot({"ybar interval", fill="blue!85", "forget plot"}, Table(h)))
pgfsave("tikz/ensemble_perf.tikz", plt_histogram)
pgfsave("tmp/ensemble_perf.pdf", plt_histogram)

In [89]:
# Average EFE of ensemble as a function of time
efes = vcat(experiment_1["efe_vals"]
            ,experiment_2["efe_vals"]
            )
efe_means = mean(efes,dims=1)[:]
avg_efe = @pgf Axis(
    {
        title="Average EFE of ensemble",
        xlabel="Time Index",
        ylabel="EFE",
        grid = "minor",
        view = (0, 90), 
        colormap = "{blackwhite}{color=(black) color=(yellow)}",
        no_marks,
        x_post_scale = 1.5
    },
    Plot(Coordinates(1:T, efe_means))
)
pgfsave("tikz/avg_efe.tikz", avg_efe)
pgfsave("tmp/avg_efe.pdf", avg_efe)